In [1]:
import json
import sys
import Globals
from Containers.AdjacencyMatrix import AdjacencyMatrix
from Parsers.MatlabNodeParser import MatlabNodeParser

from scipy.sparse import csgraph
from scipy.sparse.csgraph import depth_first_order, csgraph_from_dense, depth_first_tree


# Picks file to run
var = input("Which test file? " + 
            "1) SetOne "+ 
            "2) SetTwo "+ 
            "3) SetThree "+ 
            "4) Test "+ 
            "5) T ")
if var == 1:
  file_name = "." + Globals.TESTING_DIR  + "SetOne.mat"
  var_name = "SetOne"
elif var == 2:
  file_name = "." + Globals.TESTING_DIR + "SetTwo.mat"
  var_name = "SetTwo"
elif var == 3:
  file_name = "." + Globals.TESTING_DIR + "SetThree.mat"
  var_name = "SetThree"
elif var == 4:
  file_name = "." + Globals.TESTING_DIR + "Test.mat"
  var_name = "Test"
elif var == 5:
  file_name = "." + Globals.TESTING_DIR + "T.mat"
  var_name = "T"
else:
  print("Bad input = SetOne")
  file_name = "SetOne.mat"
  var_name = "SetOne"

# File loaded in
parsed_data = MatlabNodeParser(file_name, var_name)
try:
  parsed_data.load_data()
except IOError:
    print("Terminating...")
    sys.exit()
parsed_data.construct_node_container()

# Initialize AdjacencyMatrix
connect = AdjacencyMatrix(parsed_data._node_container)
connect.fill_matrix();

# csgraph depth_first_search run on matrix from AdjacencyMatrix
s_graph = csgraph_from_dense(connect._matrix)
print("***Print of compressed graph connectivity***")
print(s_graph)

# Printing each node's connectivity
print("")
print("***Printing each node and its connectivity***")
for n in connect._position_hash:
    if n._layer is 1:
        pos = connect._position_hash[n]
        print(str(n), pos)
        print(depth_first_order(s_graph, pos, True, True)[1])
        val = depth_first_order(s_graph, pos, True, True)[1]
        print('\n')

test = {}
for n in connect._nodes:
  test[str(n)] = connect._position_hash[n]

print(json.dumps(test))
print(json.dumps(val.tolist()))


Which test file? 1) SetOne 2) SetTwo 3) SetThree 4) Test 5) T 1
***Print of compressed graph connectivity***
  (0, 1)	1.0
  (0, 2)	1.0
  (1, 3)	1.0
  (2, 4)	1.0
  (2, 5)	1.0
  (3, 6)	1.0
  (4, 7)	1.0
  (5, 8)	1.0

***Printing each node and its connectivity***
('Layer: 1 Node: 1 Input Nodes: [1, 7]', 0)
[-9999     0     0     1     2     2     3     4     5]


{"Layer: 3 Node: 33 Input Nodes: [70]": 4, "Layer: 2 Node: 1 Input Nodes: [30]": 1, "Layer: 2 Node: 7 Input Nodes: [33, 40]": 2, "Layer: 1 Node: 1 Input Nodes: [1, 7]": 0, "Layer: 3 Node: 30 Input Nodes: [42]": 3, "Layer: 4 Node: 70 Input Nodes: []": 7, "Layer: 4 Node: 100 Input Nodes: []": 8, "Layer: 3 Node: 40 Input Nodes: [100]": 5, "Layer: 4 Node: 42 Input Nodes: []": 6}
[-9999, 0, 0, 1, 2, 2, 3, 4, 5]
